Домашнее задание 1

In [0]:
!pip install pandas

In [9]:
from google.colab import files

uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [0]:
import pandas as pd

weather_hist = pd.read_csv('weatherHistory.csv')
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

Проанализируйте данные и найдите самый холодный и самый ветренный дни в Сегеде за всю имеющуюся историю.

In [0]:
min_temp = weather_hist['temp'].min()
max_wind_speed = weather_hist['wind_speed'].max()
coldest_day = weather_hist.loc[weather_hist['temp'] == min_temp]['date']
windest_day = weather_hist.loc[weather_hist['wind_speed'] == max_wind_speed]['date']
print(min_temp, coldest_day)
print(max_wind_speed, windest_day)

-21.822222222222226 54847    2012-02-10 07:00:00.000 +0100
Name: date, dtype: object
63.8526 12181    2007-01-29 13:00:00.000 +0100
Name: date, dtype: object


In [0]:
weather_hist['year'] = pd.to_datetime(weather_hist['date'], utc=True).dt.year
weather_hist['month'] = pd.to_datetime(weather_hist['date'], utc=True).dt.month

Узнайте сколько раз шел дождь летом 2014 года.

In [0]:
weather_hist[(weather_hist['weather_type'] == 'rain') & (weather_hist['year'] == 2014) & (weather_hist['month'] >= 6) & (weather_hist['month'] < 9)].shape[0]

2205

Сгруппируйтей данные по годам, месяцам и неделям (по отдельности) и найдите:

1) самый ветрянный месяц

In [0]:
weather_hist.groupby(weather_hist['month'])['wind_speed'].mean().sort_values(ascending=False)

month
3     13.415825
2     12.180815
4     11.876139
1     11.515865
12    11.094578
5     10.953186
11    10.947953
10    10.008495
7      9.637124
6      9.632453
9      9.610204
8      8.942707
Name: wind_speed, dtype: float64

Сгруппируйтей данные по годам, месяцам и неделям (по отдельности) и найдите:

2) самый влажный год

In [0]:
weather_hist.groupby(weather_hist['year'])['humidity'].mean().sort_values(ascending=False)

year
2005    0.890000
2010    0.796861
2006    0.767349
2016    0.760868
2013    0.754197
2014    0.748571
2011    0.736008
2015    0.732354
2009    0.707247
2008    0.701239
2007    0.689647
2012    0.689513
Name: humidity, dtype: float64

Сгруппируйтей данные по годам, месяцам и неделям (по отдельности) и найдите:

3) неделю с самой переменчивой погодой (там, где отклонение какого-то или нескольких признаков самое большое)

In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
weather_hist['week'] = pd.to_datetime(weather_hist['date'], utc=True).dt.week
weeks = {}
for i in range(1,54):
  weeks[i] = 0
stds = weather_hist.groupby(weather_hist['week']).std()
for column in stds:
  rank = 0
  for index, row in stds[column].sort_values(ascending=False).iteritems():
    weeks[index] = weeks[index] + rank
    rank += 1
strange_week = 0
strange_rank = 1000000
for w in weeks:
  if weeks[w] < strange_rank:
    strange_week = w
    strange_rank = weeks[w]
print(strange_week)

12


Домашнее задание 2

In [3]:
uploaded = files.upload()

Saving data.csv to data.csv


В качестве признаков возьмем:
- начинается на большую букву, заканчивается на знак препинания - т.к. описание выглядит полным и законченным
- количество последовательных слов через запятую(обычно однородные прилагательные) - т.к. даюьт более красочное описание
- количество длинных слов - т.к. по идее люди не любят много длинных слов
- количество коротких слов - т.к. по идее слишком короткие лсова это тоже не очень
- количество аббревиатур=названий брендов - т.к. за них цепляется глаз и проще выбрать описание с ними

In [0]:
import re
import numpy as np
from sklearn.metrics import mean_squared_error

data = pd.read_csv('data.csv')
def count_adj(x):
  res = re.match('([a-zа-я]+,)+', x.lower())
  if res is None:
    return 0
  return len(re.match('([a-zа-я]+,)+', x.lower()).group().split(','))

data['starts_ends_properly'] = data['description'].apply(lambda x: re.match('^[A-ZА-ЯЁ].*[.!?]$', x) is not None)
data['consecutive_adj_count'] = data['description'].apply(lambda x: count_adj(x))
data['long_words'] = data['description'].apply(lambda x: len(re.findall('[a-zа-я]{15,40}', x.lower())))
data['short_words'] = data['description'].apply(lambda x: len(re.findall('[a-zа-я]{2,7}', x.lower())))
data['brand_words'] = data['description'].apply(lambda x: len(re.findall('[A-ZА-ЯЁ]+', x)))

In [0]:
X = data[['starts_ends_properly', 'consecutive_adj_count', 'long_words', 'short_words', 'brand_words']].values
y = data.deal_probability.values

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26359978859450045

Качество стало лучше, чем при использовании прошлых признаков

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
X = cv.fit_transform(data.description)
y = data.deal_probability.values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
regressor = Ridge(alpha=10)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
old_res = np.sqrt(mean_squared_error(y_test, preds))
print(old_res)

0.2512515450632489


In [0]:
cvs_mf = [CountVectorizer(max_features=1000-200, min_df=5, max_df=0.4, ngram_range=(1,2)),
       CountVectorizer(max_features=1000-100, min_df=5, max_df=0.4, ngram_range=(1,2)),
       CountVectorizer(max_features=1000+100, min_df=5, max_df=0.4, ngram_range=(1,2)),
       CountVectorizer(max_features=10000+200, min_df=5, max_df=0.4, ngram_range=(1,2))]
cvs_min_df = [CountVectorizer(max_features=1000, min_df=1, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=2, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=7, max_df=0.4, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=10, max_df=0.4, ngram_range=(1,2))]
cvs_max_df = [CountVectorizer(max_features=1000, min_df=5, max_df=0.1, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=0.5, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=2.0, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=3.5, ngram_range=(1,2)),
              CountVectorizer(max_features=1000, min_df=5, max_df=4.0, ngram_range=(1,2))]
cvs_ngram_range = [CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,1)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,3)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,4)),
                   CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,5))]
cvs = [cvs_mf, cvs_min_df, cvs_max_df, cvs_ngram_range]
for cvs_group in cvs:
  for cv in cvs_group:
    X = cv.fit_transform(data.description)
    y = data.deal_probability.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    regressor = Ridge(alpha=10)
    regressor.fit(X_train, y_train)
    preds = regressor.predict(X_test)
    res = np.sqrt(mean_squared_error(y_test, preds))  
    print(res, res <= old_res)
  print('\n')

0.2515666907240369 False
0.25142841678202654 False
0.2512676856760321 False
0.25513504648301877 False


0.2512515450632489 True
0.2512483226513516 True
0.2512515450632489 True
0.25123968780690215 True
0.2512483226513516 True


0.25156431949139557 False
0.2512515450632489 True
0.2512515450632489 True
0.2512515450632489 True
0.2512515450632489 True


0.2512641479584974 False
0.2512515450632489 True
0.2512626583953009 False
0.2512519560170297 False
0.2512559216510182 False




In [15]:
from sklearn.linear_model import SGDRegressor
from scipy.sparse import hstack
cv = CountVectorizer(max_features=1000, min_df=7, max_df=0.4, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=1000, min_df=5,analyzer='char', ngram_range=(3,3), max_df=0.4)

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
regressor = SGDRegressor(max_iter=50, alpha=0.00001)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
old_res = np.sqrt(mean_squared_error(y_test, preds))
print(old_res)

0.24974338598050602


In [23]:
regressors = [
              SGDRegressor(max_iter=50, alpha=0.00001, loss='huber'),
              SGDRegressor(max_iter=50, alpha=0.00001, loss='epsilon_insensitive'),
              SGDRegressor(max_iter=50, alpha=0.00001, loss='squared_epsilon_insensitive'),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.2),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.4),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.6),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=0.8),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='l1', l1_ratio=1),
              SGDRegressor(max_iter=50, alpha=0.00001, penalty='elasticnet'),
]
for regressor in regressors:
  regressor.fit(X_train, y_train)
  preds = regressor.predict(X_test)
  res = np.sqrt(mean_squared_error(y_test, preds))
  print(res, res <= old_res)

0.26921955278520165 False
0.2584085013431037 False
0.2543628650126234 False
0.24964955491879928 True
0.24974323436426202 True
0.24971695503709235 True
0.24964046976666732 True
0.24965998117324237 True
0.24977672408763674 False
0.24968248946779822 True


In [22]:
regressors = [
              SGDRegressor(max_iter=25, alpha=0.00001),
              SGDRegressor(max_iter=75, alpha=0.00001),
              SGDRegressor(max_iter=100, alpha=0.00001),
              SGDRegressor(max_iter=125, alpha=0.00001),
              SGDRegressor(max_iter=150, alpha=0.00001),
              SGDRegressor(max_iter=300, alpha=0.00001),
              SGDRegressor(max_iter=600, alpha=0.00001),
              SGDRegressor(max_iter=1000, alpha=0.00001),
              SGDRegressor(max_iter=50, alpha=0.000001),
              SGDRegressor(max_iter=50, alpha=0.000005),
              SGDRegressor(max_iter=50, alpha=0.00003),
              SGDRegressor(max_iter=50, alpha=0.00005),
              SGDRegressor(max_iter=50, alpha=0.0001),
              SGDRegressor(max_iter=300, alpha=0.000005, penalty='l1', l1_ratio=1),
]
for regressor in regressors:
  regressor.fit(X_train, y_train)
  preds = regressor.predict(X_test)
  res = np.sqrt(mean_squared_error(y_test, preds))
  print(res, res <= old_res)

0.24986069322253207 False
0.24965634000675238 True
0.2497222348030067 True
0.24960615285599527 True
0.2496191102473483 True
0.2496439330821555 True
0.24961884981026433 True
0.2496738869764999 True
0.24987152749466388 False
0.24965149718747778 True
0.24984687020077725 False
0.24966219027779896 True
0.25003744360503083 False
0.24967080585723894 True


К сожалению, подобрать такие параметры, которые б улучшили результат не удалось; были перебраны различные варианты для параметров CV в отдельности, но оказалось, что только смена min_df чуть-чуть улучшила результат. При этом перебор других параметров при новом min_df не дал ощутимых результатов. Смена параметров регрессора немного улучшает результат, но обогнать 0.248 не может :С